In [1]:
%pip install -r /home/aalab/Desktop/tripoFT/TripoSR/requirements.txt

  Cloning https://github.com/tatsy/torchmcubes.git to /tmp/pip-req-build-krfspzb8
  Running command git clone --filter=blob:none --quiet https://github.com/tatsy/torchmcubes.git /tmp/pip-req-build-krfspzb8
  fatal: unable to access 'https://github.com/tatsy/torchmcubes.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/tatsy/torchmcubes.git /tmp/pip-req-build-krfspzb8 did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/tatsy/torchmcubes.git /tmp/pip-req-build-krfspzb8 did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)

2.3.0+cu121
12.1


In [3]:
import sys
import os

# Add the TripoSR directory to the Python path
sys.path.append(os.path.abspath('TripoSR'))

# Import the TSR class
from tsr.system import TSR
from omegaconf import OmegaConf


In [4]:
import sys
import os
import torch
from omegaconf import OmegaConf

# Add the TripoSR directory to the Python path
sys.path.append(os.path.abspath('TripoSR'))

# Import the TSR class
from tsr.system import TSR

# Load the configuration file
config_path = 'TripoSR/config.yaml'
cfg = OmegaConf.load(config_path)

# Manually set `tokenizer.num_channels` before initializing other components
cfg.backbone.in_channels = cfg.tokenizer.num_channels

# Initialize the TSR model with the loaded and updated configuration
model = TSR(cfg=cfg)

/home/aalab/Desktop/tripoFT/myenv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/aalab/Desktop/tripoFT/myenv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [5]:
import sys
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class MeshDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.files = [f for f in os.listdir(data_dir) if f.endswith('.npz')]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.files[idx])
        data = np.load(file_path)
        vertices = data['vertices']
        faces = data['faces']

        # Convert to torch tensors
        vertices = torch.tensor(vertices, dtype=torch.float32)
        faces = torch.tensor(faces, dtype=torch.long)  # Indices should be long integers

        return vertices, faces

def custom_collate_fn(batch):
    vertices_list = [item[0] for item in batch]
    faces_list = [item[1] for item in batch]

    # Padding vertices
    max_vertices = max([v.shape[0] for v in vertices_list])
    padded_vertices = [torch.cat([v, torch.zeros(max_vertices - v.shape[0], 3)], dim=0) for v in vertices_list]
    
    # Padding faces
    max_faces = max([f.shape[0] for f in faces_list])
    padded_faces = [torch.cat([f, torch.zeros(max_faces - f.shape[0], 3, dtype=torch.long)], dim=0) for f in faces_list]

    vertices_tensor = torch.stack(padded_vertices)
    faces_tensor = torch.stack(padded_faces)

    return vertices_tensor, faces_tensor

# Paths to your data directories
train_dir = '/home/aalab/Desktop/tripoFT/split_data/train'
val_dir = '/home/aalab/Desktop/tripoFT/split_data/val'
test_dir = '/home/aalab/Desktop/tripoFT/split_data/test'

# Create dataset instances
train_dataset = MeshDataset(train_dir)
val_dataset = MeshDataset(val_dir)
test_dataset = MeshDataset(test_dir)

# Create data loaders with custom collate function
batch_size = 4  # Adjust as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=custom_collate_fn)

# Example of iterating through the train_loader
for vertices, faces in train_loader:
    print(vertices.shape, faces.shape)
    # Add your training code here
    break


/home/aalab/Desktop/tripoFT/myenv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/aalab/Desktop/tripoFT/myenv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/aalab/Desktop/tripoFT/myenv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/aalab/Desktop/tripoFT/myenv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


torch.Size([4, 2980976, 3]) torch.Size([4, 993662, 3])


In [6]:
import torch
import torch.nn as nn

class ChamferDistance(nn.Module):
    def forward(self, points1, points2):
        batch_size, num_points, _ = points1.size()
        dist1 = torch.cdist(points1, points2)
        dist2 = torch.cdist(points2, points1)
        
        loss1 = dist1.min(dim=2)[0].mean(dim=1)
        loss2 = dist2.min(dim=2)[0].mean(dim=1)
        
        loss = (loss1 + loss2).mean()
        return loss

# Example usage
loss_fn = ChamferDistance()


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a loss function using Chamfer Distance
class ChamferDistance(nn.Module):
    def forward(self, points1, points2):
        batch_size, num_points, _ = points1.size()
        dist1 = torch.cdist(points1, points2)
        dist2 = torch.cdist(points2, points1)

        loss1 = dist1.min(dim=2)[0].mean(dim=1)
        loss2 = dist2.min(dim=2)[0].mean(dim=1)

        loss = (loss1 + loss2).mean()
        return loss

loss_fn = ChamferDistance()

# Define an optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adjust learning rate as needed

# Function to transform model output to point cloud
def transform_output_to_point_cloud(output):
    batch_size, _, D, H, W = output.shape
    # Example transformation: Flatten the volumetric grid and sample points
    output_flat = output.view(batch_size, 3, -1)  # Flatten the spatial dimensions
    output_points = output_flat.permute(0, 2, 1)  # Reorder to shape [batch_size, num_points, 3]
    return output_points


/home/aalab/Desktop/tripoFT/myenv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [8]:
# Wrapper function to handle model call with device
def call_model_with_device(model, vertices, device):
    return model(vertices, device)

In [9]:
# Define the train function with memory optimization
def train(model, train_loader, val_loader, loss_fn, optimizer, num_epochs, device):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for i, (vertices, faces) in enumerate(train_loader):
            # Move data to the appropriate device
            vertices, faces = vertices.to(device), faces.to(device)

            # Reduce batch size for memory optimization
            batch_size = min(vertices.size(0), 1)
            vertices = vertices[:batch_size]
            faces = faces[:batch_size]

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            model_output = call_model_with_device(model, vertices, device)
            outputs = transform_output_to_point_cloud(model_output)

            # Ensure output shape matches vertices shape
            if outputs.shape[1] != vertices.shape[1]:
                outputs = outputs[:, :vertices.shape[1], :]

            # Calculate loss
            loss = loss_fn(outputs, vertices)
            loss.backward()

            # Optimize
            optimizer.step()
            
            # Accumulate running loss
            running_loss += loss.item()

            # Clear GPU cache to free up memory
            torch.cuda.empty_cache()

        # Print training loss
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}')

        # Validation step
        model.eval()
        with torch.no_grad():
            val_loss = 0
            for vertices, faces in val_loader:
                vertices, faces = vertices.to(device), faces.to(device)
                model_output = call_model_with_device(model, vertices, device)
                outputs = transform_output_to_point_cloud(model_output)

                if outputs.shape[1] != vertices.shape[1]:
                    outputs = outputs[:, :vertices.shape[1], :]

                val_loss += loss_fn(outputs, vertices).item()
            val_loss /= len(val_loader)
        
        # Print validation loss
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}')

        # Clear GPU cache after validation
        torch.cuda.empty_cache()

# Example usage
num_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
train(model, train_loader, val_loader, loss_fn, optimizer, num_epochs, device)


In [ ]:
# Testing function
def test(model, test_loader, loss_fn, device):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for vertices, faces in test_loader:
            vertices, faces = vertices.to(device), faces.to(device)
            model_output = call_model_with_device(model, vertices, device)
            outputs = transform_output_to_point_cloud(model_output)

            if outputs.shape[1] != vertices.shape[1]:
                outputs = outputs[:, :vertices.shape[1], :]

            test_loss += loss_fn(outputs, vertices).item()
        test_loss /= len(test_loader)

    print(f'Test Loss: {test_loss:.4f}')

# Example usage
test(model, test_loader, loss_fn, device)

KeyboardInterrupt: 